In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import statsmodels.formula.api as sm
import statsmodels.api as smf
from scipy.stats import gamma,norm,rayleigh,describe,scoreatpercentile

%matplotlib inline

In [ ]:
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from utils.Run_LCA_model import Run_LCA_model
from utils.Parameter_values_dict import Parameter_names

In [3]:
buildings_path = '..\inputs\\combined_buildings_2.csv'
logistics = 'optimal'
analysis = 'Sensitivity'
acid_type = 'Sulfuric'
scenario = 'Resin'
n_collection = 1
n_regen = 20
for direction in ['plus', 'minus']:
    Result_Energy_m3 = pd.DataFrame()
    Result_GHG_m3 = pd.DataFrame()
    Result_COST_m3 = pd.DataFrame()
    for param in Parameter_names:
        Total_Energy_m3, Total_GHG_m3, Total_COST_m3, Parameters = Run_LCA_model(buildings_path, n_regen, n_collection, logistics, analysis,acid_type, scenario=scenario, parameter = param, direction = direction)
        Total_Energy_m3['param'], Total_GHG_m3['param'], Total_COST_m3['param'] = param, param, param
        Result_Energy_m3 = Result_Energy_m3.append(Total_Energy_m3)
        Result_GHG_m3 = Result_GHG_m3.append(Total_GHG_m3)
        Result_COST_m3 = Result_COST_m3.append(Total_COST_m3)

    Result_Energy_m3["sum"] = (Result_Energy_m3.sum(axis=1)) - Result_Energy_m3["n_facilities"]-Result_Energy_m3['Fertilizer offset']
    Result_GHG_m3["sum"] = (Result_GHG_m3.sum(axis=1)) - Result_GHG_m3["n_facilities"]-Result_GHG_m3['Fertilizer offset']
    Result_COST_m3["sum"] = (Result_COST_m3.sum(axis=1)) - Result_COST_m3["n_facilities"]-Result_COST_m3['Fertilizer offset']-Result_COST_m3['Labor']-Result_COST_m3['Labor_trucks']-Result_COST_m3['Labor_facility']

    if direction == 'plus':
        Result_Energy_param_plus = Result_Energy_m3[['sum', 'param']]
        Result_GHG_param_plus = Result_GHG_m3[['sum', 'param']]
        Result_COST_param_plus = Result_COST_m3[['sum', 'param']]
    if direction == 'minus':
        Result_Energy_param_minus = Result_Energy_m3[['sum', 'param']]
        Result_GHG_param_minus = Result_GHG_m3[['sum', 'param']]
        Result_COST_param_minus = Result_COST_m3[['sum', 'param']]

In [4]:
Sensitivity_Energy = pd.merge(Result_Energy_param_plus, Result_Energy_param_minus, on = 'param')
Sensitivity_GHG = pd.merge(Result_GHG_param_plus, Result_GHG_param_minus, on = 'param')
Sensitivity_COST = pd.merge(Result_COST_param_plus, Result_COST_param_minus, on = 'param')

In [5]:
Sensitivity_Energy['Sensitivity'] = abs(Sensitivity_Energy['sum_y'] - Sensitivity_Energy['sum_x'])
Sensitivity_GHG['Sensitivity'] = abs(Sensitivity_GHG['sum_y'] - Sensitivity_GHG['sum_x'])
Sensitivity_COST['Sensitivity'] = abs(Sensitivity_COST['sum_y'] - Sensitivity_COST['sum_x'])

In [6]:
Sensitivity_Energy.sort(columns='Sensitivity', ascending=False)[:8]

C:\Users\Olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,sum_x,param,sum_y,Sensitivity
0,28.592126,percent_served,26.030785,2.561341
56,35.877138,sulphuric_acid_energy,34.105488,1.771650
13,34.215719,urine_production,35.963884,1.748165
10,34.337971,adsorption_density,35.677459,1.339488
23,35.274604,transport_energy_MJ_km,34.708022,0.566582
9,34.856569,resin_lifetime,35.137286,0.280718
5,35.131028,resin_energy_MJ_kg,34.851598,0.279430
11,35.111352,time_between_catridge_regeneration,34.871275,0.240077


In [7]:
Sensitivity_COST.sort(columns='Sensitivity', ascending=False)[:8]

C:\Users\Olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,sum_x,param,sum_y,Sensitivity
0,16.459498,percent_served,14.009965,2.449533
13,21.798771,urine_production,23.564857,1.766086
75,23.189926,facility_cost_regression,21.974707,1.215219
10,22.227465,adsorption_density,22.954984,0.727519
11,22.877508,time_between_catridge_regeneration,22.287124,0.590383
61,22.754520,sulphuric_acid_cost,22.410112,0.344409
9,22.550026,resin_lifetime,22.617297,0.067271
4,22.615848,resin_cost_kg,22.548784,0.067063


In [8]:
Sensitivity_GHG.sort(columns='Sensitivity', ascending=False)[:8]

C:\Users\Olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,sum_x,param,sum_y,Sensitivity
0,5.319362,percent_served,4.705342,0.614020
13,6.666572,urine_production,7.087893,0.421321
57,7.019323,sulphuric_acid_GHG,6.687670,0.331653
10,6.733797,adsorption_density,6.979206,0.245409
24,6.921371,transport_GHG_kg_km,6.785622,0.135750
11,6.871349,time_between_catridge_regeneration,6.835644,0.035704
9,6.837255,resin_lifetime,6.871092,0.033837
6,6.870262,resin_GHG_kg_kg,6.836731,0.033532
